In [1]:
import matplotlib.pyplot as plt
import pandas as pd

data = pd.read_csv('../data/nyc_taxi.csv')
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp',inplace=True)
print data.head()
print data.tail()
print data.index.dtype,len(data)



                     value
timestamp                 
2014-07-01 00:00:00  10844
2014-07-01 00:30:00   8127
2014-07-01 01:00:00   6210
2014-07-01 01:30:00   4656
2014-07-01 02:00:00   3820
                     value
timestamp                 
2015-01-31 21:30:00  24670
2015-01-31 22:00:00  25721
2015-01-31 22:30:00  27309
2015-01-31 23:00:00  26591
2015-01-31 23:30:00  26288
datetime64[ns] 10320


In [2]:
# use 4 days to predict 1 day
import numpy as np

ONE_DAY_DATA_LEN = 48
def make_data(X, pre_days=4):
    xs = []
    ys = []
    data_len = len(X)
    for i in range(data_len-pre_days):
        x = X[i:i+pre_days]
        y = X[i+pre_days:i+pre_days+1]

        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

daily_data = data.values.flatten()

# preprocess data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
daily_data = scaler.fit_transform(daily_data).reshape(-1,ONE_DAY_DATA_LEN)

print daily_data.shape
x,y = make_data(daily_data)
print x.shape,y.shape

def split_train_test(x, y, n):
    x_train, x_test = x[0:-n], x[-n:]
    y_train, y_test = y[0:-n], y[-n:]
    return x_train, y_train, x_test, y_test

X_train, Y_train, X_test, Y_test = split_train_test(x, y, 10)
print X_train.shape, Y_train.shape, X_test.shape, Y_test.shape
_,input_len,input_dim = X_train.shape
_,output_len,output_dim = Y_train.shape
hidden_dim = output_len * 2

print X_train[0]
print Y_train[0]

(215, 48)
(211, 4, 48) (211, 1, 48)
(201, 4, 48) (201, 1, 48) (10, 4, 48) (10, 1, 48)
[[-0.61874487 -1.01029084 -1.28654908 -1.51049551 -1.63097119 -1.76744305
  -1.84007432 -1.88402777 -1.86140256 -1.87048146 -1.81903431 -1.55257553
  -1.24101042 -0.59064348 -0.18454243  0.10482979  0.40097516  0.75058522
   0.63428872  0.71614301  0.55430785  0.37215323  0.30427757  0.47922671
   0.54335551  0.5401851   0.43815545  0.62275994  0.63529748  0.78589209
   0.61137528  0.49089959  0.1571416  -0.01795165  0.29764853  0.63227118
   1.12815256  1.79566855  1.68455999  1.4074371   1.11503858  0.75750248
   1.19084026  1.3404261   1.17887916  0.95046533  0.71571068  0.14028077]
 [-0.25472384 -0.74829946 -1.09041582 -1.32877321 -1.48686152 -1.65777559
  -1.75014989 -1.81615212 -1.81110828 -1.8233576  -1.7681636  -1.53557059
  -1.20325369 -0.5850232  -0.20298847  0.22516136  0.57909471  0.69395012
   0.62895666  0.55574895  0.31321237  0.3986694   0.28756085  0.49738453
   0.5759243   0.59393801

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:646: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.r

In [3]:
# get errors

import keras
from seq2seq.models import SimpleSeq2seq, Seq2seq


X_train = X_train.astype("float32")
Y_train = Y_train.astype("float32")

model = Seq2seq(batch_input_shape=(1,input_len,input_dim),output_dim=output_dim,hidden_dim=hidden_dim,output_length=output_len)
model.compile(loss='mse', optimizer='rmsprop')
print 'compile done'



Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.
compile done


In [ ]:
for i in range(20):
    model.fit(X_train,Y_train,nb_epoch=1,batch_size=1,shuffle=False,verbose=1,validation_data=(X_test, Y_test))
    model.reset_states()

Train on 201 samples, validate on 10 samples
Epoch 1/1
201/201 [==============================] - 10s - loss: 0.8291 - val_loss: 1.0435
Train on 201 samples, validate on 10 samples
Epoch 1/1
201/201 [==============================] - 5s - loss: 0.6220 - val_loss: 0.8911
Train on 201 samples, validate on 10 samples
Epoch 1/1
201/201 [==============================] - 7s - loss: 0.4398 - val_loss: 0.8299
Train on 201 samples, validate on 10 samples
Epoch 1/1
201/201 [==============================] - 10s - loss: 0.3489 - val_loss: 0.7708
Train on 201 samples, validate on 10 samples
Epoch 1/1
201/201 [==============================] - 13s - loss: 0.3350 - val_loss: 0.7605
Train on 201 samples, validate on 10 samples
Epoch 1/1
201/201 [==============================] - 13s - loss: 0.3351 - val_loss: 0.7561
Train on 201 samples, validate on 10 samples
Epoch 1/1
201/201 [==============================] - 11s - loss: 0.3254 - val_loss: 0.7514
Train on 201 samples, validate on 10 samples
Epoch

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
pred = []
for x in X_test:
    pred.append(model.predict(np.array([x])))
pred = np.array(pred)
print pred.shape
pred = scaler.inverse_transform(pred.reshape(-1,1))
truth = scaler.inverse_transform(Y_test.reshape(-1,1))

from sklearn import metrics

print metrics.explained_variance_score(truth,pred)
print metrics.r2_score(truth,pred)
print metrics.mean_absolute_error(truth,pred),truth.mean(),truth.std()

plt.plot(pred,label='pred')
plt.plot(truth,label='truth')
plt.legend()
plt.show()

(10, 1, 1, 48)
0.449149784739
0.417516984421
4295.83871001 13846.8083333 7897.20557781
